In [26]:
%load_ext autoreload
%autoreload 2

In [27]:
from load_data import load_data, load_item, gen_full_data, gen_train_eval
from conf import opt
import pandas as pd

In [39]:
import time, datetime

In [28]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'
import faiss
import numpy as np

In [29]:
from utils.evaluate import evaluate
from datetime import datetime
import time

In [30]:
train_data = load_data(opt.data.base_dir+opt.data.train_file, opt.phrase)
test_data = load_data(opt.data.base_dir+opt.data.test_file, opt.phrase)
qtime_data = load_data(opt.data.base_dir+opt.data.predict_file, opt.phrase, qtime=True)
user_df = pd.read_csv(opt.data.base_dir+opt.data.user_feat, names=opt.data.user_cols, header=None)
item_df = load_item(opt.data.base_dir+opt.data.item_feat)

data/underexpose_train/underexpose_train_click-0.csv
data/underexpose_train/underexpose_train_click-1.csv
data/underexpose_train/underexpose_train_click-2.csv
data/underexpose_test/underexpose_test_click-0.csv
data/underexpose_test/underexpose_test_click-1.csv
data/underexpose_test/underexpose_test_click-2.csv
data/underexpose_test/underexpose_test_qtime-0.csv
data/underexpose_test/underexpose_test_qtime-1.csv
data/underexpose_test/underexpose_test_qtime-2.csv


In [31]:
train_data['flag'] = 0
test_data['flag'] = 1
data = gen_full_data([train_data, test_data])

In [33]:
data[data['user_id'] == 11]

,user_id,item_id,qtime,flag,timestamp,time_diff
10025,11,88058,0.983740,1,1.560652e+09,-1.000000
9908,11,66082,0.983740,1,1.560653e+09,741.645294
8462,11,29755,0.983741,1,1.560653e+09,575.500734
15760,11,78804,0.983770,1,1.560700e+09,46394.367025
18254,11,43074,0.983830,1,1.560796e+09,95790.345706
20397,11,75757,0.983830,1,1.560796e+09,8.006967
581562,11,23323,0.983868,0,1.560856e+09,60051.249669
500363,11,22113,0.983868,0,1.560856e+09,55.047896
588771,11,20317,0.983868,0,1.560856e+09,313.272574
4188,11,64689,0.983868,1,1.560856e+09,70.060959


In [34]:
data['time_str'] = data['timestamp'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

In [35]:
data['pharse_id'] = data['user_id'] % 11

In [36]:
cnt_df = data.groupby(['user_id'])['item_id'].count().reset_index()
cnt_df.columns = ['user_id', 'item_count']

In [37]:
'''
answer_df = eval_data.drop_duplicates(subset=['user_id'], keep='last')
item_df = pd.read_csv(opt.data.base_dir+"item_count.csv")
df_merge = pd.merge(answer_df, item_df)
df_merge[['user_id', 'item_id', 'item_degree']].to_csv(opt.data.local_eval_file,index=False, header=None)
evaluate(sub_path, opt.data.local_eval_file, opt.phrase)
'''

'\nanswer_df = eval_data.drop_duplicates(subset=[\'user_id\'], keep=\'last\')\nitem_df = pd.read_csv(opt.data.base_dir+"item_count.csv")\ndf_merge = pd.merge(answer_df, item_df)\ndf_merge[[\'user_id\', \'item_id\', \'item_degree\']].to_csv(opt.data.local_eval_file,index=False, header=None)\nevaluate(sub_path, opt.data.local_eval_file, opt.phrase)\n'

In [38]:
data.head()

,user_id,item_id,qtime,flag,timestamp,time_diff,time_str,pharse_id
19829,1,78142,0.983742,0,1.560655e+09,-1.000000,2019-06-16 11:15:24,1
236132,1,26646,0.983757,0,1.560679e+09,23854.755656,2019-06-16 17:52:59,1
20480,1,89568,0.983763,0,1.560690e+09,10758.360684,2019-06-16 20:52:17,1
19709,1,76240,0.983770,0,1.560701e+09,11097.655900,2019-06-16 23:57:15,1
108033,1,87533,0.983790,0,1.560731e+09,30260.329047,2019-06-17 08:21:35,1


In [43]:
t = (2020, 4, 10, 0, 0, 0, 0, 0, 0)
time_end = time.mktime(t)
def time_info(time_delta):
    timestamp = time_end * time_delta
    struct_time = time.gmtime(timestamp)
    return (struct_time.tm_wday, struct_time.tm_hour, struct_time.tm_min)

In [46]:
data['day'],  data['hour'], data['minute'] = zip(*data['qtime'].apply(time_info))

In [51]:
feature_cols = ['user_id', 'item_id', 'pharse_id', 'day', 'hour', 'minute']

In [95]:
feature_map = data[feature_cols].max()+1
feature_map = feature_map.to_dict()

In [8]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *

In [65]:
from tensorflow.keras import Model


In [68]:
from tensorflow.keras.optimizers import Adam

In [146]:
class SampledSoftmaxLoss(object):
  """ The loss function implements the Dense layer matmul and activation
  when in training mode.
  """
  def __init__(self, model, num_classes, num_sampled):
    self.model = model
    output_layer = model.layers[-1]
    self.input = output_layer.input
    self.weights = output_layer.weights
    self.num_classes = num_classes
    self.num_sampled = num_sampled

  def loss(self, y_true, y_pred, **kwargs):
    labels = tf.argmax(y_true, axis=1)
    labels = tf.expand_dims(labels, -1)
    loss = tf.nn.sampled_softmax_loss(
        weights=self.weights[0],
        biases=self.weights[1],
        labels=labels,
        inputs=self.input,
        num_sampled = self.num_sampled,
        num_classes = self.num_classes,
        #partition_strategy = "div",
    )
    return loss

In [159]:
k_latent = 8
embedding_reg = 0.0002
kernel_reg = 0.1

class SampledSoftmaxLoss(object):
  """ The loss function implements the Dense layer matmul and activation
  when in training mode.
  """
  def __init__(self, model, num_classes, num_sampled):
    self.model = model
    output_layer = model.layers[-1]
    self.input = output_layer.input
    self.weights = output_layer.weights
    self.num_classes = num_classes
    self.num_sampled = num_sampled

  def loss(self, y_true, y_pred, **kwargs):
    labels = tf.argmax(y_true, axis=1)
    labels = tf.expand_dims(labels, -1)
    loss = tf.nn.sampled_softmax_loss(
        weights=self.weights[0],
        biases=self.weights[1],
        labels=labels,
        inputs=self.input,
        num_sampled = self.num_sampled,
        num_classes = self.num_classes,
        #partition_strategy = "div",
    )
    return loss

def get_embed(x_input, x_size, k_latent):
    if x_size > 0: #category
        embed = Embedding(x_size, k_latent, input_length=1, 
                          embeddings_regularizer=l2(embedding_reg))(x_input)
        embed = Flatten()(embed)
    else:
        embed = Dense(k_latent, kernel_regularizer=l2(embedding_reg))(x_input)
    return embed

def build_model(X, feature_map, nums_neg=5):
    
    f_size = [dim for name, dim in feature_map.items()]
    dim_input, dim_output = len(f_size), (nums_neg+1) * 2
    
    input_x = [Input(shape=(1,)) for i in range(dim_input)]     
    biases = [get_embed(x, size, 1) for (x, size) in zip(input_x, f_size)]
    factors = [get_embed(x, size, k_latent) for (x, size) in zip(input_x, f_size)]
    s = Add()(factors)
    diffs = [Subtract()([s, x]) for x in factors]
    dots = [Dot(axes=1)([d, x]) for d,x in zip(diffs, factors)]
    x = Concatenate()(biases + dots)
    #x = BatchNormalization()(x)
    output = Dense(dim_output, activation='relu', kernel_regularizer=l2(kernel_reg))(x)
    model = Model(inputs=input_x, outputs=[output])
    
    loss_calculator = SampledSoftmaxLoss(model, feature_map['item_id'], nums_neg)
    opt = Adam(clipnorm=0.5)
    model.compile(optimizer=opt, loss=loss_calculator.loss)
    output_f = factors + biases
    model_features = Model(inputs=input_x, outputs=output_f)
    return model, model_features

In [160]:
model, model_features = build_model(data, feature_map, nums_neg=5)

In [131]:
import numpy as np
nb_classes = feature_map['item_id']
targets = data['item_id'].values.reshape(-1)
one_hot_targets = np.eye(nb_classes, dtype=np.int8)[targets]
r = [data[col] for col, dim in feature_map.items()]

In [ ]:
model.fit(x=r, y=y)